In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Substance_Use_Disorder/GSE172166'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Identifying a novel biological mechanism for alcohol addiction associated with circRNA networks acting as potential miRNA sponges."
!Series_summary	"Our lab and others have shown that chronic alcohol use leads to gene and miRNA expression changes across the mesocorticolimbic (MCL) system. Circular-RNA (circRNA) are noncoding RNA that form closed-looped structures and are reported to alter gene expression through miRNA sequestration, thus providing a potentially novel neurobiological mechanism for the development of alcohol dependence (AD). Genome-wide expression of circRNA was assessed in nucleus accumbens (NAc) from 32 AD matched cases/controls. Significant circRNA (unadj. p≤0.05) were identified via regression and clustered in circRNA networks via WGCNA. CircRNA interactions with previously generated mRNA and miRNA were detected via correlation and bioinformatic analyses. MatrixeQTL was used to identify eQTLs affecting circRNA expression. Genomi

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if the dataset contains gene expression data
is_gene_available = True  # Given the background information, the dataset assesses genome-wide expression.

# Variable Availability and Data Type Conversion
# Mapping for Substance_Use_Disorder
if 0 in [0]:
    trait_row = 0  # matching diagnosis: Alcohol, diagnosis: Control 
else:
    trait_row = None

# Mapping for age
if 1 in [0, 1, 2, 3, 4, 5]:
    age_row = 1  # matching age: values
else:
    age_row = None

# Mapping for gender
# No explicit information about gender, consider it unavailable
gender_row = None

# Convert data type functions
def convert_trait(value):
    # Map "Alcohol" to 1 and "Control" to 0
    val = value.split(":")[1].strip()
    if val == "Alcohol":
        return 1
    elif val == "Control":
        return 0
    return None

def convert_age(value):
    # Age is continuous
    try:
        return float(value.split(":")[1].strip())
    except ValueError:
        return None

def convert_gender(value):
    # Gender data is not available
    return None

# Save Metadata
save_cohort_info('GSE172166', './preprocessed/Substance_Use_Disorder/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Substance_Use_Disorder', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Substance_Use_Disorder/trait_data/GSE172166.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM5243269': [1.0, 61.0], 'GSM5243270': [0.0, 44.0], 'GSM5243271': [0.0, 62.0], 'GSM5243272': [1.0, 56.0], 'GSM5243273': [0.0, 63.0], 'GSM5243274': [1.0, 42.0], 'GSM5243275': [0.0, 46.0], 'GSM5243276': [0.0, 56.0], 'GSM5243277': [1.0, 52.0], 'GSM5243278': [0.0, 43.0], 'GSM5243279': [1.0, 59.0], 'GSM5243280': [1.0, 56.0], 'GSM5243281': [1.0, 54.0], 'GSM5243282': [1.0, 46.0], 'GSM5243283': [1.0, 39.0], 'GSM5243284': [1.0, 73.0], 'GSM5243285': [0.0, 56.0], 'GSM5243286': [0.0, 50.0], 'GSM5243287': [1.0, 63.0], 'GSM5243288': [1.0, 50.0], 'GSM5243289': [1.0, 50.0], 'GSM5243290': [1.0, 51.0], 'GSM5243291': [1.0, 55.0], 'GSM5243292': [0.0, 47.0], 'GSM5243293': [0.0, 55.0], 'GSM5243294': [1.0, 53.0], 'GSM5243295': [0.0, 82.0], 'GSM5243296': [0.0, 64.0], 'GSM5243297': [1.0, 73.0], 'GSM5243298': [0.0, 73.0], 'GSM5243299': [0.0, 57.0], 'GSM5243300': [0.0, 59.0]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ASCRP3000001', 'ASCRP3000002', 'ASCRP3000003', 'ASCRP3000004',
       'ASCRP3000005', 'ASCRP3000006', 'ASCRP3000007', 'ASCRP3000008',
       'ASCRP3000009', 'ASCRP3000010', 'ASCRP3000011', 'ASCRP3000012',
       'ASCRP3000013', 'ASCRP3000014', 'ASCRP3000015', 'ASCRP3000016',
       'ASCRP3000017', 'ASCRP3000018', 'ASCRP3000019', 'ASCRP3000020'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ASCRP3000001', 'ASCRP3000002', 'ASCRP3000003', 'ASCRP3000004', 'ASCRP3000005'], 'circRNA': ['hsa_circRNA_082501', 'hsa_circRNA_407247', 'hsa_circRNA_007418', 'hsa_circRNA_407121', 'hsa_circRNA_407087'], 'TYPE': ['circRNA', 'circRNA', 'circRNA', 'circRNA', 'circRNA'], 'BUILD': ['HG19', 'HG19', 'HG19', 'HG19', 'HG19'], 'SEQUENCE': ['AAAAAAACAACCAAAAAATGTTCAACAGCATGAGAAGGTTCAGAAAGCCAGTACAGAGGG', 'AAAAAACGAAGAAAAAGAGACACCCAGCTCACCTCCAAGTTTGCCTGCAGGAGCCGGCTC', 'AAAAAACTAGAAATGTGTTCAGAAATTAAAGGTCCACAGAAGGAGGGCCTGTCCTCCCAA', 'AAAAAAGAAGTCCTCGGATACGCTGAAACTGCAGAAGAAAGCAAAAAAAGGTGGGACCAG', 'AAAAAAGGAACAGGAAAACTACATCTCAGACCCAACTATCTTATGGAGGGCTTATCCCCA']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Define the keys for identifiers and gene symbols based on previous step outputs.
identifier_key = 'ID'
gene_symbol_key = 'circRNA'

# 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function from the library.
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library, and name the resulting gene expression dataframe "gene_data".
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Substance_Use_Disorder/gene_data/GSE172166.csv'
normalized_gene_data.to_csv(gene_csv_path)

# Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Substance_Use_Disorder')

# Save the cohort information.
save_cohort_info('GSE172166', './preprocessed/Substance_Use_Disorder/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Substance_Use_Disorder/GSE172166.csv'
    unbiased_merged_data.to_csv(csv_path)


No gene data in the dataframe
